Evaluation

Import environment variables

In [11]:
import dotenv
import os

dotenv.load_dotenv()


True

Load docs

In [12]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "test_data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

docs[0]

Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2023-07-15T11:40:07+05:30', 'source': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'file_path': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'total_pages': 24, 'format': 'PDF 1.7', 'title': 'Microsoft Word - Introduction to relational databases', 'author': 'Administrator', 'subject': '', 'keywords': '', 'moddate': '2023-07-15T11:40:07+05:30', 'trapped': '', 'modDate': "D:20230715114007+05'30'", 'creationDate': "D:20230715114007+05'30'", 'page': 0}, page_content='Introduction to relational databases \n \nIntroduction to relational databases \n \n\uf0b7 A relational database is based on the relational model and uses a collection of \ntables to represent both data and the relationship among those data. \n\uf0b7 It also includes DDL and DML  \n\uf0b7 Relational database was originally defined by Edger Codd at IBM Research \ncenter in 1970. \n\uf0b7 In relational database, user only needs to understand logical str

choose Llm

In [67]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.embeddings import OpenAIEmbeddings
import openai

evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
openai_client = openai.OpenAI()
generator_embeddings = OpenAIEmbeddings(client=openai_client)

/tmp/ipykernel_82574/2774485547.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
/tmp/ipykernel_82574/2774485547.py:7: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))


Generate testset

In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=5)

Applying EmbeddingExtractor:   0%|          | 0/23 [00:00<?, ?it/s]/root/py/proj/backend/.venv/lib/python3.13/site-packages/ragas/testset/transforms/base.py:188: UserWarning: Using sync embedding model OpenAIEmbeddings in async context. This may impact performance. Consider using an async-compatible embedding model for better performance.
  property_name, property_value = await self.extract(node)
Generating Samples: 100%|██████████| 6/6 [00:04<00:00,  1.46it/s]


In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What significant development in relational dat...,[Introduction to relational databases \n \nInt...,The relational database was originally defined...,single_hop_specific_query_synthesizer
1,What is the significance of a primary key in a...,[ A stored procedure is a high-end database t...,A primary key is a field that uniquely identif...,single_hop_specific_query_synthesizer
2,What is the significance of the full outer joi...,[<1-hop>\n\n Types of outer join \no Left out...,The full outer join in SQL is significant beca...,multi_hop_abstract_query_synthesizer
3,What are the extended relational algebra opera...,[<1-hop>\n\nIII) Extended Relational Algebra o...,The extended relational algebra operations inc...,multi_hop_abstract_query_synthesizer
4,Wht is the city of the cust_name in the Deposi...,[<1-hop>\n\n Consider two table (Depositor & ...,The city of the cust_name in the Depositor tab...,multi_hop_specific_query_synthesizer
5,How do you identify candidate keys in the Empl...,[<1-hop>\n\n Candidate keys are selected from...,To identify candidate keys in the Employee tab...,multi_hop_specific_query_synthesizer


Prepere data

In [29]:
from agent.agent import tutor_agent
from langchain_core.messages import HumanMessage

In [75]:
# from agent.utils.tools import read_from_temp

# eval_contexts = read_from_temp()
# eval_contexts

res = tutor_agent.invoke({"messages": [HumanMessage("How to use key?")], "retrieved_docs": []})
res["messages"]

# res["retrieved_docs"]


[HumanMessage(content='How to use key?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 194, 'total_tokens': 213, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_1f35c1788c', 'id': 'chatcmpl-CSrlgIpNi3NNwUX89aJEHbMZ6MWqo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--ff8640fb-4788-4428-ad5e-3562d9b30302-0', tool_calls=[{'name': 'knowledge_base_search', 'args': {'query': 'How to use key'}, 'id': 'call_zFt7Zgi1TtNsQuGXXnod66JV', 'type': 'tool_call'}], usage_metadata={'input_tokens': 194, 'output_tokens': 19, 'total_tokens': 213, 'input_token_details': {'audio': 0,

In [65]:
for test_row in dataset:
    query = test_row.eval_sample.user_input
    response = tutor_agent.invoke({"messages": [HumanMessage(query)]})

    # test_row.eval_sample.retrieved_docs = response["retrieved_docs"]
    test_row.eval_sample.response = response["messages"][-1].content
    

In [66]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

Run evaluation

In [70]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

result = evaluate(dataset=evaluation_dataset,metrics=[FactualCorrectness()],llm=evaluator_llm)
result

Evaluating: 100%|██████████| 6/6 [00:25<00:00,  4.26s/it]


{'factual_correctness(mode=f1)': 0.5883}